## **Sentiment Analysis- Arabic News** 
### *Evaluating Arabic News Attitudes towards America through NLP Analysis and a Predictive Classification Model*

***

by: Andrew Rachuig, Kyle Green, Mindy Shiben, and Paige Rackley

### Introduction : 
*** 
The United States has made itself a key player in the Middle East by using its diplomatic, economic, and military power in support of national and international interests. Since the 9/11 terrorist attacks on U.S. soil, one of the most consequntial events in the modern world, tensions between the two regions have been undoubtedly high. With remarkable communication and technology enhancements in the 21st century, we've become more aware of biases portrayed in mainstream media, and Middle Eastern media is no exception. Two-thirds of Arab nationals overall say they trust mass media such as newspapers, TV, and radio to report news fully, fairly, and accurately (mideastmedia.org, 2017). Through examining almost half a million Arabic news articles between 2000 and 2014 from 10 different Middle Eastern news outlets, we can see how different factors (such as international events) may drive overall sentiment and sentiment towards America. Having this insight may inform policy makers and influence decision making moving forward. This project delivers an integral piece of the puzzle of America's foreign policy towards the world.

### Project Goals:
*** 
The goal 

### Summary of Findings & Recommendations:
***
- words 

### Acquiring the Data

### Prepare the Data

### Explore

#### Initial Hypothesis and Questions: **** We need to choose if we want to remove any of these questions or add any new **
    - What’s the relationship between article sentiment and world events?
    - What is the relationship between news sources and the target? 
    - What is the relationship between country of source and target?
    - What is the relationsheep between tag and target?
    - Is there a relationship between time and sentiment?
    - Is there a relationship between sentiment of the headline and sentiment of text?

### Example of how we can format our questions:


Question:


Visual:


Stat testing if needed


Takeway

#### Exploration Summary: Summarizes all takeways from visuals, stat tests and questions. 

### Model

Depending on our steps here, we can show prep such as splitting, explanations for specific features we want to use and why, etc.


What are we focusing on? What is our baseline? Why is that our baseline?


If we will be testing on different models, show results - train then takeways

Validate - validate checks for overfitting if needed and helps us decide which model to take into test if the results are very close during train. If we need to show validate, its best to explain what we're looking for and what our final decision is 

Test - one final model and results of model.

### In conclusion (or better way of saying that)

Where we talk about our results compared to baseline, did we beat it, what kind of model, etc.

#### With more time:
   - Next steps we may expand on next week, things we want to look further into
   
#### Recommendations:
    - Can kind of depend what to put here.